In [59]:
import pandas as pd
from glob import glob
import numpy as np
from easydict import EasyDict as edict
import os
import time
import sys
from tabulate import tabulate

sys.path.insert(0,os.path.expanduser('~/git/torchseg'))
print(sys.path)

from utils.config import load_config
from utils.summary_to_csv import config_to_log,load_log,edict_to_pandas

def summary(rootpath,tags,filter_str=None,):
    config_files=glob(os.path.join(rootpath,'**','config.txt'),recursive=True)
    if filter_str is not None:
        config_files=[f for f in config_files if f.find(filter_str)>=0]
    
    tasks=pd.DataFrame()
    for cfg in config_files:
        log=config_to_log(cfg)
        if log is not None:
            ed=load_config(cfg)
            metrics=load_log(log,tags)
            for key,value in metrics.items():
                ed[key]=value
            
            ed['dir']=cfg
            task=edict_to_pandas(ed)
            tasks=tasks.append(task,ignore_index=True,sort=False)
        else:
            print('cannot find log file for',cfg)
            
    return tasks

def dump(tags=['train/fmeasure','val/fmeasure'],
        rootpath=os.path.expanduser('~/tmp/logs/motion'),
        notes=['motion_fcn','motion_sparse','motion_psp'],
        note_gtags=[['upsample_layer','freeze_layer','ignore_outOfRoi'],
                    ['upsample_layer','freeze_layer','sparse_ratio'],
                   ['upsample_layer','freeze_layer','ignore_outOfRoi']],
        descripe=['epoch'],
        delete_nan=False,
        dump_group=False,
        dump_dir=True):
    dir_tags=[tags[1],'dir']
    for note,ntag in zip(notes,note_gtags):
        show_tags=[]
        show_tags=tags.copy()
        group_tags=['note']+ntag
        show_tags+=group_tags
        show_tags+=descripe
        tasks=summary(rootpath,tags,note)
        if tasks.empty:
            print(note,ntag,'task is empty')
            continue

        # remove empty dirs
        if delete_nan:
            dirs=tasks[tasks[tags[1]].isna()]['dir'].tolist()
            for dir in dirs:
                print(os.path.dirname(dir))
                os.system('rm -rf {}'.format(os.path.dirname(dir)))

        #print(tasks[show_tags].groupby(group_tags).max().to_string())
        if dump_group:
            print(tabulate(tasks[show_tags].groupby(group_tags).mean().sort_values(tags[1]),tablefmt='pipe',headers='keys'))
            print('\n')
            print(tabulate(tasks[[tags[1]]+group_tags].groupby(group_tags).agg([np.mean,np.std,np.max]),tablefmt='pipe',headers='keys'))
            print('\n')
        print(tabulate(tasks[show_tags].sort_values(tags[1]),tablefmt='pipe',headers='keys'))
        print('\n')
        if dump_dir:
            print(tabulate(tasks[dir_tags].sort_values(tags[1]),tablefmt='pipe',headers='keys'))
            print('\n')

['/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '', '/home/yzbx/git/torchseg/notebooks', '/home/yzbx/git/gnu/models/research', '/home/yzbx/git/gnu/models/research/slim', '/home/yzbx/bin/miniconda3/envs/new/lib/python36.zip', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/lib-dynload', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/cityscapesscripts-1.0.0-py3.6-linux-x86_64.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/IPython/extensions', '/home/yzbx/.ipython']


In [8]:
rootpath=os.path.expanduser('~/tmp/logs/motion')
tags=['train/acc','val/acc']
notes=['/stn_u','wo_stn_u']
note_gtags=[['upsample_layer','freeze_layer'],
            ['upsample_layer','freeze_layer']]
describe=[]
for note,ntag in zip(notes,note_gtags):
    show_tags=[]
    show_tags=tags.copy()
    group_tags=['note']+ntag
    show_tags+=group_tags
    show_tags+=['epoch']
    tasks=summary(rootpath,tags,note)
    if tasks.empty:
        print(note,ntag,'task is empty')
        continue
        
    #print(tasks[show_tags].groupby(group_tags).max().to_string())
    print(tabulate(tasks[show_tags].groupby(group_tags).mean().sort_values('val/acc'),tablefmt='pipe',headers='keys'))
    print('\n')
    print(tabulate(tasks[['val/acc']+group_tags].groupby(group_tags).agg([np.mean,np.std,np.max]),tablefmt='pipe',headers='keys'))
    print('\n')
    print(tabulate(tasks[show_tags].sort_values('val/acc'),tablefmt='pipe',headers='keys'))
    print('\n')

|                                 |   train/acc |   val/acc |   epoch |
|:--------------------------------|------------:|----------:|--------:|
| ('stn_u3f{freeze_layer}', 3, 3) |    0.916383 |  0.854048 |      30 |
| ('stn_u5f{freeze_layer}', 5, 3) |    0.959955 |  0.854574 |      30 |
| ('stn_u3f{freeze_layer}', 3, 0) |    0.987225 |  0.860692 |      30 |
| ('stn_u4f{freeze_layer}', 4, 3) |    0.966873 |  0.863473 |      30 |
| ('stn_u3f{freeze_layer}', 3, 2) |    0.967766 |  0.863873 |      30 |
| ('stn_u4f{freeze_layer}', 4, 2) |    0.980876 |  0.865827 |      30 |
| ('stn_u4f{freeze_layer}', 4, 1) |    0.985273 |  0.869544 |      30 |
| ('stn_u3f{freeze_layer}', 3, 1) |    0.985943 |  0.869708 |      30 |
| ('stn_u5f{freeze_layer}', 5, 2) |    0.977819 |  0.869732 |      30 |
| ('stn_u4f1', 4, 1)              |    0.986909 |  0.875642 |      30 |
| ('stn_u5f{freeze_layer}', 5, 1) |    0.987404 |  0.880334 |      30 |
| ('stn_u4f{freeze_layer}', 4, 0) |    0.987871 |  0.88127  |   

In [46]:
rootpath=os.path.expanduser('~/tmp/logs/motion/motion_unet')
notes=['/deconv','/vgg','/pre_vgg','/atrous_vgg']
note_gtags=[['upsample_layer','freeze_layer','ignore_outOfRoi'],
            ['upsample_layer','freeze_layer','ignore_outOfRoi'],
           ['upsample_layer','freeze_layer','ignore_outOfRoi'],
           ['upsample_layer','freeze_layer']]
dump(rootpath=rootpath,notes=notes,note_gtags=note_gtags,delete_nan=False)

|    |   train/fmeasure |   val/fmeasure | note      |   upsample_layer |   freeze_layer | ignore_outOfRoi   |   epoch |
|---:|-----------------:|---------------:|:----------|-----------------:|---------------:|:------------------|--------:|
|  1 |         0.649625 |       0.520457 | deconv2_5 |                2 |              3 | True              |      30 |
|  3 |         0.707233 |       0.522959 | deconv3_5 |                3 |              3 | True              |      30 |
|  0 |         0.747155 |       0.586299 | deconv1_5 |                1 |              3 | True              |      30 |
|  2 |         0.822215 |       0.606581 | deconv1_4 |                1 |              3 | True              |      30 |


|    |   val/fmeasure | dir                                                                                         |
|---:|---------------:|:--------------------------------------------------------------------------------------------|
|  1 |       0.520457 | /home/yzbx/t

In [30]:
print(tasks.columns)

Index(['backbone_name', 'dataset', 'epoch', 'frame_gap', 'freeze_layer',
       'init_lr', 'log_dir', 'motion_loss_weight', 'net_name', 'note',
       'pose_mask_reg', 'save_model', 'stn_loss_weight', 'stn_object',
       'train_path', 'upsample_layer', 'val_path', 'dir', 'root_path',
       'use_part_number', 'test_path'],
      dtype='object')


In [53]:
tags=['train/fmeasure','val/fmeasure']
rootpath=os.path.expanduser('~/tmp/logs/motion')
notes=['motion_fcn','motion_sparse','motion_psp']
note_gtags=[['use_none_layer'],
            ['use_none_layer','sparse_ratio'],
           ['use_none_layer',]]
dump(tags=tags,rootpath=rootpath,notes=notes,note_gtags=note_gtags,descripe=['epoch','net_name'],delete_nan=False)

|    |   train/fmeasure |   val/fmeasure | note                  | use_none_layer   |   epoch | net_name    |
|---:|-----------------:|---------------:|:----------------------|:-----------------|--------:|:------------|
|  1 |         0.954051 |       0.567287 | atrous_vgg11_u3_d4_f2 | True             |      30 | motion_fcn2 |
|  8 |         0.954877 |       0.580052 | atrous_vgg11_u3_d4_f0 | True             |      30 | motion_fcn2 |
|  4 |         0.955238 |       0.594882 | atrous_vgg11_u3_d4_f1 | True             |      30 | motion_fcn2 |
|  9 |         0.956377 |       0.606298 | test                  | True             |      30 | motion_fcn2 |
|  2 |         0.956975 |       0.609185 | atrous_vgg11_u5_d4_f2 | True             |      30 | motion_fcn2 |
|  0 |         0.955239 |       0.618403 | atrous_vgg11_u5_d4_f1 | True             |      30 | motion_fcn2 |
|  3 |         0.95643  |       0.625173 | atrous_vgg11_u4_d4_f2 | True             |      30 | motion_fcn2 |
|  7 |    

|    |   train/fmeasure |   val/fmeasure | note                  | use_none_layer   |   epoch | net_name   |
|---:|-----------------:|---------------:|:----------------------|:-----------------|--------:|:-----------|
|  2 |         0.554843 |       0.550867 | sparse_vgg11_u3_d5_f0 | True             |      30 | motion_psp |
|  9 |         0.579434 |       0.566858 | sparse_vgg11_u3_d5_f1 | True             |      30 | motion_psp |
|  7 |         0.921005 |       0.56746  | sparse_vgg11_u3_d5_f1 | True             |      30 | motion_psp |
|  1 |         0.894513 |       0.586117 | sparse_vgg11_u3_d5_f0 | True             |      30 | motion_psp |
|  8 |         0.914304 |       0.58972  | sparse_vgg11_u3_d5_f1 | True             |      30 | motion_psp |
|  0 |         0.902043 |       0.599361 | sparse_vgg11_u3_d5_f0 | True             |      30 | motion_psp |
|  6 |         0.886629 |       0.610465 | sparse_vgg11_u3_d5_f1 | True             |      30 | motion_psp |
|  4 |         0.95

In [62]:
tags=['train/fmeasure','val/fmeasure']
rootpath=os.path.expanduser('~/tmp/logs/motion')
notes=['task001','task004','task005','task006']
note_gtags=[['use_none_layer','use_part_number'],
            ['sparse_ratio','upsample_layer'],
           ['dataset'],
           ['sparse_ratio']]
dump(tags=tags,rootpath=rootpath,notes=notes,note_gtags=note_gtags,descripe=['epoch','net_name'],delete_nan=False)

|    |   train/fmeasure |   val/fmeasure | note          | use_none_layer   |   use_part_number |   epoch | net_name    |
|---:|-----------------:|---------------:|:--------------|:-----------------|------------------:|--------:|:------------|
|  2 |         0.987435 |       0.672596 | task001_4000  | False            |              4000 |      30 | motion_unet |
|  0 |         0.98914  |       0.672686 | task001_2000  | False            |              2000 |      30 | motion_unet |
|  1 |         0.98997  |       0.684593 | task001_1000  | False            |              1000 |      30 | motion_unet |
|  4 |         0.987208 |       0.70522  | task001_T2000 | True             |              2000 |      30 | motion_unet |
|  5 |         0.987001 |       0.712173 | task001_T1000 | True             |              1000 |      30 | motion_unet |
|  3 |         0.987396 |       0.726716 | task001_T4000 | True             |              4000 |      30 | motion_unet |


|    |   val/fmeasure 